Loading the model in 

In [104]:
import torch
import pandas as pd
from torch import nn 
import joblib

# from common_utils import MLPBuilder
from torch.utils.data import DataLoader, TensorDataset
import torch.optim as optim
from sklearn.model_selection import train_test_split
modelname = 'DNN_model_with_features_SWLab[512, 512, 256, 256]'
modellayer = [512, 512, 256, 256]
no_features=11


In [105]:
class MLPBuilder(nn.Module):

    def __init__(self, no_features, layers):
        super().__init__()
        layer_list = []
        
        # Input layer
        layer_list.append(nn.Linear(no_features, layers[0]))
        layer_list.append(nn.ReLU())
        layer_list.append(nn.Dropout(p=0.2))

        # Hidden layers
        for i in range(len(layers) - 1):
            layer_list.append(nn.Linear(layers[i], layers[i+1]))
            layer_list.append(nn.ReLU())
            layer_list.append(nn.Dropout(p=0.2))

        # Define the MLP stack as a sequential model
        self.mlp_stack = nn.Sequential(*layer_list)

        # Output layer, 1 outputs
        self.output_x = nn.Linear(layers[-1], 8) 
        self.output_y = nn.Linear(layers[-1], 8) 


        self._initialize_weights() 
        
    def forward(self, x):
        features = self.mlp_stack(x)
        logits_x = self.output_x(features)
        logits_y = self.output_y(features)
        return logits_x, logits_y
    
    def _initialize_weights(self):
        for layer in self.mlp_stack:
            if isinstance(layer, nn.Linear):
                # Use Kaiming initialization for ReLU activations
                nn.init.kaiming_uniform_(layer.weight, nonlinearity='relu')
                if layer.bias is not None:
                    nn.init.zeros_(layer.bias)

In [106]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = MLPBuilder(no_features, layers=modellayer)
model.load_state_dict(torch.load('models/Grid_based/'+ modelname +'.pth', map_location=device))
model.to(device)
model.eval()  

C:\Users\randa\AppData\Local\Temp\ipykernel_12104\1835588017.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('models/Grid_based/'+ model

MLPBuilder(
  (mlp_stack): Sequential(
    (0): Linear(in_features=11, out_features=512, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.2, inplace=False)
    (3): Linear(in_features=512, out_features=512, bias=True)
    (4): ReLU()
    (5): Dropout(p=0.2, inplace=False)
    (6): Linear(in_features=512, out_features=256, bias=True)
    (7): ReLU()
    (8): Dropout(p=0.2, inplace=False)
    (9): Linear(in_features=256, out_features=256, bias=True)
    (10): ReLU()
    (11): Dropout(p=0.2, inplace=False)
  )
  (output_x): Linear(in_features=256, out_features=8, bias=True)
  (output_y): Linear(in_features=256, out_features=8, bias=True)
)

## Loading test data in 

In [107]:


train_file_path = './Data/25-02-10/cleaned_df_features.csv'

test_file_path = './Data/25-02-04/cleaned_df_features.csv'

model_path = './models/Grid_based'
scaler_X = joblib.load(model_path +'/scaler_X.pkl')
# scaler_Y = joblib.load(model_path +'/scaler_Y.pkl')

train_cleaned_df = pd.read_csv(train_file_path)

In [108]:
train_cleaned_df

,Tx_0 RSSI,Tx_1 RSSI,Tx_2 RSSI,Tx_3 RSSI,Tx_4 RSSI,Tx_5 RSSI,Tx_6 RSSI,Tx_7 RSSI,X_Coord,Y_Coord,RSSI_Mean,RSSI_Variance,RSSI_Median
0,-61,-74,-79,127,-81,-78,127,-84,0,0,-25.375,8892.267857,-76.0
1,-61,-73,-78,-71,127,-78,127,127,0,0,2.500,10656.571429,-66.0
2,-62,-75,-78,-72,-79,127,-84,127,0,0,-24.500,8784.285714,-73.5
3,-61,-73,-80,-71,-83,-78,127,127,0,0,-24.000,8730.571429,-72.0
4,-61,-75,-78,-71,-84,-78,-84,-86,0,0,-77.125,68.125000,-78.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6395,-79,-69,-81,-74,-70,-61,-78,-68,6,0,-72.500,45.428571,-72.0
6396,-77,-69,-77,-71,-70,-61,-79,-67,6,0,-71.375,36.553571,-70.5
6397,-77,-69,-81,-73,-70,-61,-79,-66,6,0,-72.000,46.571429,-71.5
6398,-80,-69,-77,-72,-71,-61,-76,-67,6,0,-71.625,37.125000,-71.5


In [109]:
# batt_columns = [col for col in cleaned_df.columns if 'Batt' in col]
# time_columns = [col for col in cleaned_df.columns if 'Time' in col]
# columns_to_drop = batt_columns+time_columns
# RSSI_columns = cleaned_df.drop(columns=columns_to_drop)


# #Train the model without NAN values
# # cleaned_df = RSSI_columns.dropna()

# #Convert NaN values to 0
# cleaned_df = RSSI_columns.fillna(0)

In [110]:
# cleaned_df

# Evaluation on original dataset

In [111]:
if no_features == 8:
    X_train = train_cleaned_df[['Tx_0 RSSI', 'Tx_1 RSSI', 'Tx_2 RSSI', 'Tx_3 RSSI', 'Tx_4 RSSI', 'Tx_5 RSSI', 'Tx_6 RSSI', 'Tx_7 RSSI']]
if no_features == 11:
    X_train = train_cleaned_df[['Tx_0 RSSI', 'Tx_1 RSSI', 'Tx_2 RSSI', 'Tx_3 RSSI', 'Tx_4 RSSI', 'Tx_5 RSSI', 'Tx_6 RSSI', 'Tx_7 RSSI', 'RSSI_Mean', 'RSSI_Variance', 'RSSI_Median']]

Y_train = train_cleaned_df[['X_Coord', 'Y_Coord']]

X_train_scaled = scaler_X.transform(X_train)

Y_test_array = Y_train.to_numpy()

X_train_tensor = torch.tensor(X_train_scaled, dtype=torch.float32)
Y_train_tensor = torch.tensor(Y_test_array, dtype=torch.float32)  

train_dataset = TensorDataset(X_train_tensor, Y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=False)


In [112]:
def evaluate_model(model, dataloader):
    model.eval()  
    total_loss = 0.0
    total_displacement = 0.0
    total_samples = 0
    criterion_x = nn.CrossEntropyLoss()
    criterion_y = nn.CrossEntropyLoss()

    with torch.no_grad():
        for inputs, targets in dataloader:
            inputs, targets = inputs.to(device), targets.to(device)

            logits_x, logits_y = model(inputs)  
            
            target_x = targets[:, 0].long()
            target_y = targets[:, 1].long()

            loss_x = criterion_x(logits_x, target_x)
            loss_y = criterion_y(logits_y, target_y)

            total_loss += (loss_x.item() + loss_y.item()) * targets.size(0)
            
            # Compute displacement error (L2 distance)
            pred_x = torch.argmax(logits_x, dim=1)
            pred_y = torch.argmax(logits_y, dim=1)
            displacement = torch.sqrt((pred_x - target_x) ** 2 + (pred_y - target_y) ** 2)
            total_displacement += displacement.sum().item()
            
            total_samples += targets.size(0)

    avg_loss = total_loss / total_samples
    avg_displacement = total_displacement / total_samples  # Average displacement error

    return avg_loss, avg_displacement


In [113]:
avg_loss, displacement_loss = evaluate_model(model, train_loader)

# Print results
print(f"Test MSE Loss: {avg_loss:.6f}")
print(f"Test Displacement Loss: {displacement_loss:.6f}") 

Test MSE Loss: 12.045263
Test Displacement Loss: 3.310755


# Implementing transfer learning in test environment

In [114]:
test_cleaned_df = pd.read_csv(test_file_path)

In [115]:
if no_features == 8:    
    X_test = test_cleaned_df[['Tx_0 RSSI', 'Tx_1 RSSI', 'Tx_2 RSSI', 'Tx_3 RSSI', 'Tx_4 RSSI', 'Tx_5 RSSI', 'Tx_6 RSSI', 'Tx_7 RSSI']]
if no_features == 11: 
    X_test = test_cleaned_df[['Tx_0 RSSI', 'Tx_1 RSSI', 'Tx_2 RSSI', 'Tx_3 RSSI', 'Tx_4 RSSI', 'Tx_5 RSSI', 'Tx_6 RSSI', 'Tx_7 RSSI', 'RSSI_Mean', 'RSSI_Variance', 'RSSI_Median']]

Y_test = test_cleaned_df[['X_Coord', 'Y_Coord']]

X_test, X_val, Y_test, Y_val  = train_test_split(X_test, Y_test, test_size=0.2, random_state=42)


X_val_scaled = scaler_X.transform(X_val)
X_test_scaled = scaler_X.transform(X_test)
Y_test_scaled = Y_test.to_numpy()
Y_val_scaled = Y_val.to_numpy()

# Convert to PyTorch tensors
X_val_tensor = torch.tensor(X_val_scaled, dtype=torch.float32)
Y_val_tensor = torch.tensor(Y_val_scaled, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test_scaled, dtype=torch.float32)
Y_test_tensor = torch.tensor(Y_test_scaled, dtype=torch.float32)

val_dataset = TensorDataset(X_val_tensor, Y_val_tensor)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=True)

# Create DataLoader for the test set
test_dataset = TensorDataset(X_test_tensor, Y_test_tensor)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False)


In [116]:
avg_loss, displacement_loss = evaluate_model(model, test_loader)

# Print results
print(f"Test MSE Loss: {avg_loss:.6f}")
print(f"Test Displacement Loss: {displacement_loss:.6f}") 
before_training_loss = displacement_loss

Test MSE Loss: 0.166119
Test Displacement Loss: 0.117488


In [117]:
for param in model.mlp_stack.parameters():
    param.requires_grad = False  

# Ensure output layers are trainable
for param in model.output_x.parameters():
    param.requires_grad = True
for param in model.output_y.parameters():
    param.requires_grad = True

# Move model to device
model.to(device)


for name, param in model.named_parameters():
    print(name, param.requires_grad)
    
# Define optimizer for only trainable parameters
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=0.001)


mlp_stack.0.weight False
mlp_stack.0.bias False
mlp_stack.3.weight False
mlp_stack.3.bias False
mlp_stack.6.weight False
mlp_stack.6.bias False
mlp_stack.9.weight False
mlp_stack.9.bias False
output_x.weight True
output_x.bias True
output_y.weight True
output_y.bias True


In [118]:
import time

num_epochs = 100
patience = 5  
best_loss = float('inf')
epochs_no_improve = 0
start_time = time.time()
for epoch in range(num_epochs):
    model.train()
    total_loss = 0.0
    total_samples = 0

    for inputs, targets in test_loader:
        inputs, targets = inputs.to(device), targets.to(device)

        optimizer.zero_grad()  
        logits_x, logits_y = model(inputs)  # Forward pass
        
        target_x = targets[:, 0].long()  # Convert targets to integer indices
        target_y = targets[:, 1].long()

        loss_x = criterion(logits_x, target_x)
        loss_y = criterion(logits_y, target_y)
        
        loss = loss_x + loss_y  # Total loss
        
        loss.backward()  # Backpropagation
        optimizer.step()  

        total_loss += loss.item() * targets.size(0)  
        total_samples += targets.size(0)

    avg_loss = total_loss / total_samples  
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.6f}")

    # Early stopping logic
    if avg_loss < best_loss:
        best_loss = avg_loss
        epochs_no_improve = 0  # Reset counter if loss improves
    else:
        epochs_no_improve += 1

    if epochs_no_improve >= patience:
        print(f"Early stopping triggered after {epoch+1} epochs.")
        break  # Stop training if no improvement for `patience` epochs

total_time = time.time() - start_time  # Calculate total training time
print(f"Transfer learning complete. Total training time: {total_time:.2f} sec")

print("Transfer learning complete.")
# torch.save(model.state_dict(), "models/Regression_based/DNN_model_[256, 512, 512, 128]_finetuned.pth")


Epoch [1/100], Loss: 0.479942
Epoch [2/100], Loss: 0.446272
Epoch [3/100], Loss: 0.456034
Epoch [4/100], Loss: 0.453443
Epoch [5/100], Loss: 0.449935
Epoch [6/100], Loss: 0.436179
Epoch [7/100], Loss: 0.443318
Epoch [8/100], Loss: 0.437160
Epoch [9/100], Loss: 0.438418
Epoch [10/100], Loss: 0.429600
Epoch [11/100], Loss: 0.433203
Epoch [12/100], Loss: 0.460600
Epoch [13/100], Loss: 0.452337
Epoch [14/100], Loss: 0.444046
Epoch [15/100], Loss: 0.428899
Epoch [16/100], Loss: 0.425206
Epoch [17/100], Loss: 0.424378
Epoch [18/100], Loss: 0.439710
Epoch [19/100], Loss: 0.425111
Epoch [20/100], Loss: 0.418756
Epoch [21/100], Loss: 0.445554
Epoch [22/100], Loss: 0.436135
Epoch [23/100], Loss: 0.443854
Epoch [24/100], Loss: 0.433148
Epoch [25/100], Loss: 0.434706
Early stopping triggered after 25 epochs.
Transfer learning complete. Total training time: 2.84 sec
Transfer learning complete.


In [119]:
avg_loss, displacement_loss = evaluate_model(model, val_loader)

# Print results
print(f"Test MSE Loss: {avg_loss:.6f}")
print(f"Test Displacement Loss: {displacement_loss:.6f}")
after_training_loss = displacement_loss 

Test MSE Loss: 0.792164
Test Displacement Loss: 0.508155


In [120]:
print(f"Before Transfer learning: {before_training_loss:.6f}")
print(f"After Transfer learning: {after_training_loss:.6f}")

Before Transfer learning: 0.117488
After Transfer learning: 0.508155
